In [1]:
import math

class Node:
    def __init__(self, r = False):
        self.keys = []
        self.pointers = []
        self.data = []
        self.r = r # This variable indicates whether the node is root or not.
        
class BTree:
    def __init__(self, t, r = False): # t is maximum degree.
        self.root = Node(r)
        self.t = t
        self.min = math.ceil(t/2)-1 # This variable indicates minimum number of keys in the node.
        self.max = t-1 # This variable indicates maximum number of keys in the node.
        
    # search function works recursively.
    def search(self, key):
        # use the binary search
        left = 0
        right = len(self.root.keys) - 1
        while left <= right:
            mid = (left + right)//2
            if key < self.root.keys[mid]:
                right = mid - 1
            elif key > self.root.keys[mid]:
                left = mid + 1
            else:
                return self.root.data[mid] # return key
        
        if self.root.pointers:
            if right == len(self.root.keys) - 1:
                return self.root.pointers[-1].search(key)
            else:
                return self.root.pointers[left].search(key)
        else:
            return None
        
    def insert_key(self, key, value):
        # This part finds insertion position recursively using binary search.
        l = 0
        r = len(self.root.keys) - 1
        while l <= r:
            mid = (l + r)//2
            if key < self.root.keys[mid]:
                r = mid - 1
            elif key > self.root.keys[mid]:
                l = mid + 1

        if self.root.pointers:
            if r == len(self.root.keys) - 1:
                result = self.root.pointers[-1].insert_key(key, value)
            else:
                result = self.root.pointers[l].insert_key(key, value)


        # This part is insertion process
        if not self.root.pointers:
            if len(self.root.keys) > 0:
                self.root.keys.insert(l, key)
                self.root.data.insert(l, value)
            else:
                self.root.keys.append(key)
                self.root.data.append(value)
        elif result:
            self.root.keys.insert(l, result[0])
            self.root.data.insert(l, result[3])
            self.root.pointers[l] = result[1]
            self.root.pointers.insert(l+1,result[2])
        else:
            return None # This indicates that there is insertion key.

        if len(self.root.keys) > self.max:
            if self.root.r == False:
                return self.split()
            else: # case : this node is root
                self.root.r = False
                mid, left, right, mid_data = self.split()
                temp = BTree(self.t, True)
                temp.root.keys = [mid]
                temp.root.data = [mid_data]
                temp.root.pointers = [left, right]
                self.root = temp.root

        return None
    
    def split(self):
        left = BTree(self.t) # create new node
        right = BTree(self.t) # create new node
        left.root.keys = self.root.keys[:self.min]
        left.root.data = self.root.data[:self.min]
        right.root.keys = self.root.keys[self.min+1:]
        right.root.data = self.root.data[self.min+1:]
        mid = self.root.keys[self.min]
        mid_data = self.root.data[self.min]
        if self.root.pointers: # move the pointers
            left.root.pointers = self.root.pointers[:self.min+1]
            right.root.pointers = self.root.pointers[self.min+1:]
        
        return mid, left, right, mid_data
    
    def delete_key(self, key):
        l = 0
        r = len(self.root.keys) - 1
        varfind = 0 # This variable indicates whether the key was found or not
        while l <= r:
            mid = (l+r)//2
            if key == self.root.keys[mid]:
                varfind = 1
                break
            elif key < self.root.keys[mid]:
                r = mid - 1
            else:
                l = mid + 1
                
        if varfind == 0: # Not found case
            if self.root.pointers: # recursively works.
                if r == len(self.root.keys) - 1:
                    result = self.root.pointers[-1].delete_key(key)
                else:
                    result = self.root.pointers[l].delete_key(key)
            else:
                return None # This indicates that there is not deletion key.
        else: # Found case
            if self.root.pointers: # Found in the internal node
                idx = self.root.keys.index(key)

                # Case I : left tree's biggest key move
                left_tree = self.root.pointers[idx]
                while left_tree.root.pointers:
                    left_tree = left_tree.root.pointers[-1]
                if len(left_tree.root.keys) > self.min:
                    temp = left_tree.root.keys.pop()
                    self.root.keys[idx] = temp
                    temp = left_tree.root.data.pop()
                    self.root.data[idx] = temp
                    return None # It means that no more rotation or merging is required.

                # Case II : right tree's smallest key move
                right_tree = self.root.pointers[idx+1]
                while right_tree.root.pointers:
                    right_tree = right_tree.root.pointers[0]
                if len(right_tree.root.keys) > self.min:
                    temp = right_tree.root.keys.pop(0)
                    self.root.keys[idx] = temp
                    temp = right_tree.root.data.pop(0)
                    self.root.data[idx] = temp
                    return None # It means that no more rotation or merging is required.

                # Case III : Not Case I and Not Case II --> left tree is modified.
                temp = left_tree.root.keys.pop()
                self.root.keys[idx] = temp
                temp = left_tree.root.data.pop()
                self.root.data[idx] = temp
                temp = left_tree.root.keys[0]
                siblings = self.find_siblings(temp)
                if siblings[0] and len(siblings[0].root.keys) > self.min: # left -> right rotation case
                    siblings[3].rotate(siblings[2], True)
                    return None
                elif siblings[1] and len(siblings[1].root.keys) > self.min: # right -> left rotation case
                    siblings[3].rotate(siblings[2], False)
                    return None
                elif siblings[0]: # merging case with left node
                    siblings[3].merge(siblings[2], True)
                    if len(siblings[3].root.keys) < self.min:
                        return siblings[3].root.keys[0]
                    else:
                        return None
                elif siblings[1]: # merging case with right node
                    siblings[3].merge(siblings[2], False)
                    if len(siblings[3].root.keys) < self.min:
                        return siblings[3].root.keys[0]
                    else:
                        return None
            else: # deletion key is in the leaf node.
                idx = self.root.keys.index(key)
                self.root.keys.pop(idx)
                self.root.data.pop(idx)
                if self.root.r: # if that is root node
                    return None
                else:
                    if len(self.root.keys) >= self.min:
                        return None
                    else:
                        return self.root.keys[0]
                
        if not self.root.r: # If this node is not root
            if result:
                siblings = self.find_siblings(result)
                
                if siblings[0] and len(siblings[0].root.keys) > self.min: # left -> right rotation case
                    siblings[3].rotate(siblings[2], True)
                    return None
                elif siblings[1] and len(siblings[1].root.keys) > self.min: # right -> left rotation case
                    siblings[3].rotate(siblings[2], False)
                    return None
                elif siblings[0]: # merging case with left node
                    siblings[3].merge(siblings[2], True)
                    if len(siblings[3].root.keys) < self.min:
                        return siblings[3].root.keys[0]
                    else:
                        return None
                elif siblings[1]: # merging case with right node
                    siblings[3].merge(siblings[2], False)
                    if len(siblings[3].root.keys) < self.min:
                        return siblings[3].root.keys[0]
                    else:
                        return None
            else:
                return None
        else: # If this node is root,
            if result:
                siblings = self.find_siblings(result)
                while True:
                    if siblings[0] and len(siblings[0].root.keys) > self.min:
                        siblings[3].rotate(siblings[2], True)
                        return None
                    elif siblings[1] and len(siblings[1].root.keys) > self.min:
                        siblings[3].rotate(siblings[2], False)
                        return None
                    elif siblings[0]:
                        siblings[3].merge(siblings[2], True)
                        if len(siblings[3].root.keys) < self.min:
                            if len(siblings[3].root.keys) == 0: # If siblings[3] is root node and the node is empty, root node is modified
                                self.root.pointers[0].root.r = True
                                self.root = self.root.pointers[0].root
                            else:
                                result = siblings[3].root.keys[0]
                        else:
                            return None
                    elif siblings[1]:
                        siblings[3].merge(siblings[2], False)
                        if len(siblings[3].root.keys) < self.min:
                            if len(siblings[3].root.keys) == 0: # If siblings[3] is root node and the node is empty, root node is modified
                                self.root.pointers[0].root.r = True
                                self.root = self.root.pointers[0].root
                            else:
                                result = siblings[3].root.keys[0]
                        else:
                            return None
                    elif not siblings[0] and not siblings[1]: # If siblings[3] is root node and result is in the root node
                        return None
                    siblings = siblings[3].find_siblings(result)
            else:
                return None
    
    def find_siblings(self, key):
        left = 0
        right = len(self.root.keys) - 1
        while left <= right:
            mid = (left + right)//2
            if key < self.root.keys[mid]:
                right = mid - 1
            elif key > self.root.keys[mid]:
                left = mid + 1
            else: # If key is in the this node, this node is root node. So, it can't have the siblings.
                return None, None, left, self
        
        if self.root.pointers[0].root.pointers:
            if right == len(self.root.keys) - 1:
                if self.root.pointers[-1].search(key):
                    return self.root.pointers[-2], None, -1, self
                else:
                    return self.root.pointers[-1].find_siblings(key)
            else:
                if self.root.pointers[left].search(key): # If next node has the key
                    if left - 1 >= 0 and left + 1 < len(self.root.pointers): # Case I : it has left and right siblings
                        return self.root.pointers[left-1], self.root.pointers[left+1], left, self
                    elif left - 1 >= 0: # Case II : it has only left siblings
                        return self.root.pointers[left-1], None, left, self
                    elif left + 1 < len(self.root.pointers): # Case III : it has only right siblings
                        return None, self.root.pointers[left+1], left, self
                else:
                    return self.root.pointers[left].find_siblings(key)
        else:
            if self.root.pointers[left].search(key): # If next node has the key
                if left - 1 >= 0 and left + 1 < len(self.root.pointers):
                    return self.root.pointers[left-1], self.root.pointers[left+1], left, self
                elif left - 1 >= 0:
                    return self.root.pointers[left-1], None, left, self
                elif left + 1 < len(self.root.pointers):
                    return None, self.root.pointers[left+1], left, self
            else: # The key is not in the tree.
                return None, None, left, self
            
    def rotate(self, idx, leftside):
        if leftside: # If left -> right rotation
            if idx != -1:
                mid = self.root.keys[idx-1]
                mid_data = self.root.data[idx-1]
                left = self.root.pointers[idx-1]
                right = self.root.pointers[idx]
                right.root.keys.insert(0, mid)
                right.root.data.insert(0, mid_data)
                if left.root.pointers: # move the pointer
                    move_pointer = left.root.pointers.pop()
                    right.root.pointers.insert(0, move_pointer)
                mid = left.root.keys.pop()
                mid_data = left.root.data.pop()
                self.root.keys[idx-1] = mid # modify parent node's key
                self.root.data[idx-1] = mid_data # modify parent node's data
            else:
                mid = self.root.keys[idx]
                mid_data = self.root.data[idx]
                left = self.root.pointers[idx-1]
                right = self.root.pointers[idx]
                right.root.keys.insert(0, mid)
                right.root.data.insert(0, mid_data)
                if left.root.pointers: # move the pointer
                    move_pointer = left.root.pointers.pop()
                    right.root.pointers.insert(0, move_pointer)
                mid = left.root.keys.pop()
                mid_data = left.root.data.pop()
                self.root.keys[idx] = mid # modify parent node's key
                self.root.data[idx] = mid_data # modify parent node's data
        else: # If right -> left rotation
            mid = self.root.keys[idx]
            mid_data = self.root.data[idx]
            left = self.root.pointers[idx]
            right = self.root.pointers[idx+1]
            if right.root.pointers: # move the pointer
                move_pointer = right.root.pointers.pop(0)
                left.root.pointers.append(move_pointer)
            left.root.keys.append(mid)
            left.root.data.append(mid_data)
            mid = right.root.keys.pop(0)
            mid_data = right.root.data.pop(0)
            self.root.keys[idx] = mid # modify parent node's key
            self.root.data[idx] = mid_data # modify parent node's data
            
    def merge(self, idx, leftside):
        if leftside: # merging case with left node
            if idx != -1:
                mid = self.root.keys[idx-1]
                mid_data = self.root.data[idx-1]
                left = self.root.pointers[idx-1]
                right = self.root.pointers[idx]
                total = BTree(self.t) # total is merged result node.
                # node's keys merging step
                total.root.keys = left.root.keys
                total.root.keys.append(mid)
                total.root.keys.extend(right.root.keys)
                # data merging step
                total.root.data = left.root.data
                total.root.data.append(mid_data)
                total.root.data.extend(right.root.data)
                if left.root.pointers: # pointers merging step
                    total.root.pointers = left.root.pointers
                    total.root.pointers.extend(right.root.pointers)
                self.root.keys.remove(mid)
                self.root.data.remove(mid_data)
                self.root.pointers.insert(idx-1, total)
                self.root.pointers.remove(left)
                self.root.pointers.remove(right)
            else: # merging case with left node
                mid = self.root.keys[idx]
                mid_data = self.root.data[idx]
                left = self.root.pointers[idx-1]
                right = self.root.pointers[idx]
                total = BTree(self.t) # total is merged result node.
                # node's keys merging step
                total.root.keys = left.root.keys
                total.root.keys.append(mid)
                total.root.keys.extend(right.root.keys)
                # data merging step
                total.root.data = left.root.data
                total.root.data.append(mid_data)
                total.root.data.extend(right.root.data)
                # pointers merging step
                if left.root.pointers:
                    total.root.pointers = left.root.pointers
                    total.root.pointers.extend(right.root.pointers)
                self.root.keys.remove(mid)
                self.root.data.remove(mid_data)
                self.root.pointers.insert(idx, total)
                self.root.pointers.remove(left)
                self.root.pointers.remove(right)
        else: # merging case with right node
            mid = self.root.keys[idx]
            mid_data = self.root.data[idx]
            left = self.root.pointers[idx]
            right = self.root.pointers[idx+1]
            total = BTree(self.t) # total is merged result node.
            # node's keys merging step
            total.root.keys = left.root.keys
            total.root.keys.append(mid)
            total.root.keys.extend(right.root.keys)
            # data merging step
            total.root.data = left.root.data
            total.root.data.append(mid_data)
            total.root.data.extend(right.root.data)
            # pointers merging step
            if left.root.pointers:
                total.root.pointers = left.root.pointers
                total.root.pointers.extend(right.root.pointers)
            self.root.keys.remove(mid)
            self.root.data.remove(mid_data)
            self.root.pointers.insert(idx, total)
            self.root.pointers.remove(left)
            self.root.pointers.remove(right)

In [2]:
import pandas as pd
from tqdm import tqdm

def insertion_test(file):
    df = pd.read_csv(file, sep='\t', header=None)
    df = pd.DataFrame(list(df[1]), index = df[0], columns = ['value'])
    A = BTree(128, True)

    for k, v in tqdm(zip(df.index, df.value)):
        A.insert_key(k, v)

    print('Finish the insertion step')
    
    result = pd.DataFrame([A.search(i) for i in tqdm(df.index)], index = df.index, columns=['output'])

    print('Finish the search step')

    if sum(df.value == result.output) == len(df.index):
        print('The result is the same as the input.')

In [3]:
insertion_test('input.csv')

1000000it [00:26, 37715.62it/s]
  0%|▎                                                                       | 4804/1000000 [00:00<00:20, 47735.60it/s]

Finish the insertion step


100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:20<00:00, 48722.01it/s]


Finish the search step
The result is the same as the input.


In [4]:
insertion_test('input2.csv')

1000000it [00:26, 37625.03it/s]
  0%|▎                                                                       | 3846/1000000 [00:00<00:26, 38119.58it/s]

Finish the insertion step


100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:23<00:00, 42420.36it/s]


Finish the search step
The result is the same as the input.


In [5]:
import pandas as pd
from tqdm import tqdm

def deletion_test(input_file, delete_file, output_file):
    idf = pd.read_csv(input_file, sep='\t', header=None)
    idf = pd.DataFrame(list(idf[1]), index = idf[0], columns = ['value'])
    
    ddf = pd.read_csv(delete_file, sep='\t', header=None)
    ddf = pd.DataFrame(list(ddf[1]), index = ddf[0], columns = ['value'])
    
    odf = pd.read_csv(output_file, sep='\t', header=None)
    odf = pd.DataFrame(list(odf[1]), index = odf[0], columns = ['value'])
    
    A = BTree(128, True)

    for k, v in tqdm(zip(idf.index, idf.value)):
        A.insert_key(k, v)

    print('Finish the insertion step')

    for i in tqdm(ddf.index):
        A.delete_key(i)
    
    print('Finish the deletion step')
    
    temp = list()
    for i in tqdm(odf.index):
        result = A.search(i)
        if result:
            temp.append(result)
        else:
            temp.append('empty')

    result = pd.DataFrame(temp, index = odf.index, columns = ['output'])

    if sum(odf.value == result.output) == len(idf.index) - len(ddf.index):
        print('The result is the same as the output.')

In [6]:
deletion_test('input.csv', 'delete.csv', 'delete_result.csv')

1000000it [00:24, 40491.77it/s]
  0%|▎                                                                        | 1715/500000 [00:00<00:29, 17149.61it/s]

Finish the insertion step


  1%|▌                                                                       | 7188/1000000 [00:00<00:13, 71850.26it/s]

Finish the deletion step


100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:18<00:00, 52764.86it/s]


The result is the same as the output.


In [7]:
deletion_test('input2.csv', 'delete2.csv', 'delete_result2.csv')

1000000it [00:28, 35164.90it/s]
  0%|▎                                                                        | 2012/499724 [00:00<00:24, 19932.54it/s]

Finish the insertion step


  0%|▏                                                                       | 3123/1000000 [00:00<00:32, 31093.77it/s]

Finish the deletion step


100%|█████████████████████████████████████████████████████████████████████| 1000000/1000000 [00:26<00:00, 38431.31it/s]


The result is the same as the output.
